In [1]:
import pandas as pd
import pickle

df = pd.read_csv('CNX_HousePrice.csv')
df = df[df['Location'] == 'nimman']

y = df['price']
X = df.drop(['price', 'per_sqm_price', 'Location'], axis = 'columns')

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)

In [3]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

regressors = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Random Forest Regression': RandomForestRegressor()
}

pipelines = {}
for name, regressor in regressors.items():
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('transform', PolynomialFeatures(degree=5)),
        ('regressor', regressor)
    ])
    pipelines[name] = pipeline

results = {}
for name, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    y_pred = model.predict(X_train)
    mae = mean_absolute_error(y_train, y_pred)


    r2 = r2_score(y_train,y_pred)
    
    y_predt = model.predict(X_test)
    maet = mean_absolute_error(y_test, y_predt)


    r2t = r2_score(y_test,y_predt)
    results[name] = [mae, r2, maet, r2t] 
    

df_result = pd.DataFrame(results)
df_result = df_result.round(2)
df_result.index = ['mae_train', 'r2_train', 'mae_test', 'r2_test']
df_result

c:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.129e+13, tolerance: 6.302e+09
  model = cd_fast.enet_coordinate_descent(


,Linear Regression,Ridge Regression,Lasso Regression,Random Forest Regression
mae_train,4.145304e+05,555506.89,569967.50,454524.07
r2_train,7.600000e-01,0.66,0.64,0.77
mae_test,1.467571e+08,81711357.08,12798796.48,794534.24
r2_test,-3.398881e+05,-113625.38,-2245.31,-0.02


In [6]:
steps = [("imp_mean", SimpleImputer()), #cleaning the data by replacing missing values with avg
         ("scale", StandardScaler()), # standardize in the same range (m,var)
         ("polytransform", PolynomialFeatures(degree =2)), 
         ("regressor", RandomForestRegressor()) ]

pipeline = Pipeline(steps)

model = pipeline.fit(X_train, y_train)



In [5]:

with open('CNX_BestModel.pkl', 'wb') as model_file:
   pickle.dump(model, model_file)